In [ ]:
!pip install trl peft bitsandbytes sentencepiece wandb
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
import pandas as pd
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import KTOTrainer
import bitsandbytes as bnb
import wandb
from datasets import Dataset
import random

In [ ]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("ar-dataset-ft.csv")
df.head()

,Unnamed: 0,folder,id,lisa_sheet,mcqs,question,option_a,option_b,option_c,option_d,correct_option,can_be_guessed,completion
0,0,IC-232,OIC-232-15-B,* ''A patient with atrial fibrillation may exp...,{'question': 'What is a possible consequence o...,What is a possible consequence of atrial fibri...,Bradycardia,Hypertension,Syncope,Hyperglycemia,C,False,Question: What is a possible consequence of at...
1,1,IC-043,OIC-043-01-A,The origin of [[Abnormal genital bleeding (out...,{'question': 'What is the primary method used ...,What is the primary method used to determine t...,Patient self-assessment of symptoms,Speculum examination,Higham score,CBC (complete blood count),B,False,Question: What is the primary method used to d...
2,2,IC-307,OIC-307-04-B,"X-ray imaging is essential for diagnosis, and ...",{'question': 'In the context of diagnosing bon...,In the context of diagnosing bone lesions usin...,X-rays are sufficient to assess radiological a...,A rapidly progressive lesion typically has net...,MRI is used to determine the endomedullary ext...,All radiologically aggressive lesions are mali...,C,False,Question: In the context of diagnosing bone le...
3,3,IC-301,OIC-301-01-A,'''Tumours of the colon and rectum''' develop ...,{'question': 'Which type of colon and rectum t...,Which type of colon and rectum tumour is most ...,Hyperplastic polyps,Tubular adenoma,Juvenile polyps,Inflammatory pseudo-polyps,B,False,Question: Which type of colon and rectum tumou...
4,4,IC-096,OIC-096-01-A,Classification and definition of peripheral ne...,{'question': 'Which of the following best desc...,Which of the following best describes polyneur...,Damage to a single nerve trunk.,Lesions of the nerve plexuses.,"Damage to all nerve fibres, depending on their...",Involvement of all sensory and motor nerve roo...,C,False,Question: Which of the following best describe...


In [ ]:
df.can_be_guessed.value_counts()

,count
can_be_guessed,
False,2646
True,634


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
from pydantic import BaseModel

system_prompt = """
Based on the following educational content, generate a multiple-choice question with four answer options where only one is correct.
The question should assess understanding of the main ideas, and the options should be clear, informative, and relevant.
Ensure that the distractors (incorrect options) follow a logical but incorrect interpretation, based on common misconceptions or misunderstandings of the topic.
Ensure that the way the question is constructed does not allow a test taker with no relevant medical knowledge to identify the correct answer through clues in the phrasing, structure, answer choice formatting, or other linguistic hints.
Answer options must be as short as possible.
"""

class Message(BaseModel):
    content: str
    role: str


def train_data_format(row):
    prompt_system = Message(content=system_prompt, role='system')
    prompt_user = Message(content=row['lisa_sheet'], role="user")
    combined_prompt_array = [prompt_system.dict(), prompt_user.dict()]

    # Format completion
    completion = [Message(content=row['completion'], role="assistant").dict()]

    # Format label
    label = not bool(row['can_be_guessed'])

    return {
        "prompt": combined_prompt_array,
        "completion": completion,
        "label": label,
    }

In [ ]:
dataset_train = Dataset.from_pandas(df)
formatted_dataset_train = dataset_train.map(
    train_data_format,
    remove_columns=dataset_train.column_names
)

/usr/local/lib/python3.10/dist-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <class '__main__.Message'>.
  StockPickler.save(self, obj, save_persistent_id)
/usr/local/lib/python3.10/dist-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <class '__main__.Message'>: __main__.Message has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


Map:   0%|          | 0/3280 [00:00<?, ? examples/s]

<ipython-input-6-56442839b2cf>:19: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  combined_prompt_array = [prompt_system.dict(), prompt_user.dict()]
<ipython-input-6-56442839b2cf>:22: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  completion = [Message(content=row['completion'], role="assistant").dict()]


In [ ]:
# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    torch_dtype=torch.float16,
    load_in_4bit=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    torch_dtype=torch.float16,
    load_in_4bit=True
)
model.config.use_cache = False

peft_config = LoraConfig(
    r=8,  # Reduced rank for small dataset
    lora_alpha=16,  # Maintain scaling factor
    lora_dropout=0.1,  # Slightly increased dropout for regularization
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)

training_args = KTOConfig(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=100,  # Increased for more training iterations
    save_strategy="steps",  # Save checkpoints periodically
    save_steps=10,  # Save every 10 steps
    logging_steps=1,
    output_dir="new_model",
    optim="paged_adamw_32bit",
    warmup_steps=10,
    fp16=True,
    beta=0.1,
    max_prompt_length=2048,
    max_length=1536,
    report_to="none",
    undesirable_weight=4
)

# Create KTO trainer
kto_trainer = KTOTrainer(
    model,
    args=training_args,
    train_dataset=formatted_dataset_train,
    processing_class=tokenizer,
    peft_config=peft_config,
)

NameError: name 'AutoModelForCausalLM' is not defined